# utils

> Utility functions for system monitoring

In [ ]:
#| default_exp core.utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import socket
import webbrowser
import subprocess
import sys
from contextlib import closing
from datetime import datetime

# DaisyUI imports
from cjm_fasthtml_daisyui.components.data_display.badge import badge_colors
from cjm_fasthtml_daisyui.components.feedback.progress import progress_colors
from cjm_fasthtml_daisyui.utilities.semantic_colors import text_dui
from cjm_fasthtml_daisyui.builders.colors import ColoredUtilityDaisyUI
from cjm_fasthtml_tailwind.core.base import SingleValueUtility

In [ ]:
#| export
def find_free_port() -> int:  # An available port number
    """Find an available port."""
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

In [ ]:
#| export
def format_bytes(
    bytes_value:int  # The number of bytes to format
) -> str:  # A formatted string representation of the byte value
    """Format bytes to human readable string."""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_value < 1024.0:
            return f"{bytes_value:.1f} {unit}"
        bytes_value /= 1024.0
    return f"{bytes_value:.1f} PB"

In [ ]:
#| export
def format_bandwidth(
    bytes_per_sec:float  # The number of bytes per second to format
) -> str:  # A formatted string representation of the bandwidth value
    """Format bandwidth to human readable string."""
    if bytes_per_sec < 1024:
        return f"{bytes_per_sec:.0f} B/s"
    elif bytes_per_sec < 1024 * 1024:
        return f"{bytes_per_sec / 1024:.1f} KB/s"
    elif bytes_per_sec < 1024 * 1024 * 1024:
        return f"{bytes_per_sec / (1024 * 1024):.1f} MB/s"
    else:
        return f"{bytes_per_sec / (1024 * 1024 * 1024):.1f} GB/s"

In [ ]:
#| export
def format_uptime(
    boot_time_str:str  # Boot time in '%Y-%m-%d %H:%M:%S' format
) -> str:  # A formatted string representation of the uptime
    """Format uptime from boot time string."""
    boot_time = datetime.strptime(boot_time_str, '%Y-%m-%d %H:%M:%S')
    uptime = datetime.now() - boot_time
    days = uptime.days
    hours = uptime.seconds // 3600
    minutes = (uptime.seconds % 3600) // 60

    if days > 0:
        return f"{days}d {hours}h {minutes}m"
    elif hours > 0:
        return f"{hours}h {minutes}m"
    else:
        return f"{minutes}m"

In [ ]:
#| export
def get_progress_color(
    percent:float  # The percentage value to determine color for
) -> SingleValueUtility:  # A progress color from the DaisyUI color scheme
    """Get progress bar color based on percentage."""
    if percent < 50:
        return progress_colors.success
    elif percent < 80:
        return progress_colors.warning
    else:
        return progress_colors.error

In [ ]:
#| export
def get_temperature_color(
    temp_celsius:float,  # The temperature value in Celsius
    high:int=85,  # The threshold for high temperature
    critical:int=95  # The threshold for critical temperature
) -> ColoredUtilityDaisyUI:  # A text color from the DaisyUI semantic colors
    """Get color for temperature display."""
    if temp_celsius < 50:
        return text_dui.success
    elif temp_celsius < 70:
        return text_dui.primary
    elif temp_celsius < high:
        return text_dui.warning
    else:
        return text_dui.error

In [ ]:
#| export
def get_temperature_badge_color(
    temp_celsius:float,  # The temperature value in Celsius
    high:int=85,  # The threshold for high temperature
    critical:int=95  # The threshold for critical temperature
) -> SingleValueUtility:  # A badge color from the DaisyUI color scheme
    """Get badge color for temperature."""
    if temp_celsius < 50:
        return badge_colors.success
    elif temp_celsius < 70:
        return badge_colors.primary
    elif temp_celsius < high:
        return badge_colors.warning
    else:
        return badge_colors.error

In [ ]:
#| export
def open_browser(
    url:str  # The URL to open in the browser
) -> None:  # None
    """Open browser based on environment settings."""
    import os

    browser_mode = os.environ.get('FASTHTML_BROWSER', 'default').lower()

    if browser_mode == 'none':
        print(f"Server running at {url}")
        print("Browser auto-open disabled. Please open manually.")
        return

    if browser_mode == 'app':
        print(f"Opening in app mode at {url}")

        if sys.platform == 'linux':
            browsers = [
                ['google-chrome', '--app=' + url],
                ['chromium', '--app=' + url],
                ['firefox', '--new-window', url],
            ]

            for browser_cmd in browsers:
                try:
                    subprocess.Popen(browser_cmd,
                                   stdout=subprocess.DEVNULL,
                                   stderr=subprocess.DEVNULL)
                    return
                except FileNotFoundError:
                    continue

    print(f"Opening in browser at {url}")
    webbrowser.open(url)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()